In [80]:
import pandas as pd
from tqdm import tqdm
import os
import time
import re

from konlpy.tag import Okt, Komoran

In [61]:
okt = Okt()
komoran = Komoran()

In [67]:
# 타겟 연도
year = "2021"
# 수정하여 실행

In [68]:
file = f"{year}_news.csv"
raw = pd.read_csv(f"./news_sdc/{file}")
raw

,news_id,부동산,건설,sid2,date,content,length
0,3700782,1,0,259,20210101,[머니투데이 김철 ] [[제16회 머니투데이 경제신춘문예]]\n\n\n\n16회 ...,953
1,3700786,0,2,259,20210101,"[아시아경제 조강욱 기자] 방문규 수출입은행장은 1일 ""수출 6000억달러 탈환을 ...",1439
2,3700789,5,0,259,20210101,소비자 편익 증대 차원120건 서비스 지정\n\n\n\n[123rf][헤럴드경제=박...,1903
3,3700793,0,1,259,20210101,"4대 금융지주 인사 키워드는우리금융, 감사원 출신 신민철하나銀은 이인영 변호사 영입...",2099
4,3700795,0,1,259,20210101,"[신년사] ""K뉴딜 글로벌화·수출 중소중견기업 맞춤형 지원 최선""\n\n\n\n방문...",4968
...,...,...,...,...,...,...,...
299833,5412670,4,0,263,20211231,대출 2억 초과때 DSR 규제9억 넘는 상가주택 양도세 기준 강화‘갱신 거절’ 집주...,1700
299834,5412677,2,0,263,20211231,"홍남기 부총리 신년사…""소상공인 지원 총력""""방역제어 전제하 거시 정책수단 최적조합...",3970
299835,5412678,4,0,263,20211231,"건축물 용도·구조별 신축 표준단가 등 최신 자료 개정… 연료전지, 지열설비 등 친환...",629
299836,5412679,0,13,263,20211231,코로나19 위기 속에서도 한국이 해외건설 수주에서 선방했다. 올해 초 목표한 금액 ...,1587


In [69]:
min_length = 2000
news = raw.loc[raw["length"] >= min_length, :].copy()
news

,news_id,부동산,건설,sid2,date,content,length
3,3700793,0,1,259,20210101,"4대 금융지주 인사 키워드는우리금융, 감사원 출신 신민철하나銀은 이인영 변호사 영입...",2099
4,3700795,0,1,259,20210101,"[신년사] ""K뉴딜 글로벌화·수출 중소중견기업 맞춤형 지원 최선""\n\n\n\n방문...",4968
7,3700802,2,0,259,20210101,[머니투데이 김봉기 ] [[제16회 머니투데이 경제신춘문예]김봉기/단편소설]\n\...,20043
10,3700809,2,0,259,20210101,문재인 정부 경제관료 출신 금융 낙하산 207명 역대 최대정치인 명함 유지 수단 전...,2360
11,3700815,1,1,259,20210101,[머니투데이 이학렬 기자] [[신년사]] \n\n\n\n방문규 수출입은행장 / 사진...,5017
...,...,...,...,...,...,...,...
299809,5412460,1,0,263,20211231,"서울 오피스텔 7.03%↑…상가는 전국 평균 5.34%↑, 서울 6.74%↑가장 비...",2509
299823,5412593,3,0,263,20211231,2022 달라지는 것들\n\n\n\n사진=게티이미지뱅크세제·부동산연 매출 4000억...,6291
299828,5412633,3,0,263,20211231,◆…내년부터 근로장려금을 받을 수 있는 소득상한액이 200만원씩 오른다. 기존엔 1...,4980
299832,5412652,1,0,263,20211231,부산서 ㎡당 기준시가 가장 높은 오피스텔은 해운대 ‘경동 리인뷰 1차 102동’…상...,2177


In [83]:
# def rm_emoji(txt):
#     # 이모지 제거 / komoran은 이모지도 처리 불가
#     emoji_pattern = re.compile("["
#             u"\U0001F600-\U0001F64F"  # emoticons
#             u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#             u"\U0001F680-\U0001F6FF"  # transport & map symbols
#             u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                                "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'', txt)

In [90]:
datas = []

req = sorted(news["date"].unique())

for date in req[:3]:
    print(date, req[-1])
    words = []
    texts = news.loc[news["date"] == date, "content"].values
    
    if len(texts) >= 1:
        for txt in tqdm(texts):
            nouns = okt.nouns(txt)
            # nouns = komoran.nouns(rm_emoji(txt).replace("\n", " "))  # komoran은 다중 \n을 제대로 처리하지 못하는듯
            words.extend(nouns)

        words_vc = pd.Series(words).value_counts()
        data = words_vc.to_dict()
        data["date"] = date
        datas.append(data)

20210101 20211231


100%|██████████| 90/90 [00:40<00:00,  2.20it/s]


20210102 20211231


100%|██████████| 43/43 [00:17<00:00,  2.43it/s]


20210103 20211231


100%|██████████| 92/92 [00:34<00:00,  2.70it/s]


In [91]:
# 저장
df = pd.DataFrame(datas)
df = df.set_index("date")
df.to_csv(f"./bow_{year}.csv")

In [92]:
df

,것,등,수,주택,투자,이,금융,코로나,기업,부동산,...,군중심리,기재,집값불,누증,숙의,모형도,반납,대위,테이프,버블경제
date,,,,,,,,,,,,,,,,,,,,,
20210101,800,556,534,398,393,348,325,311,290,283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210102,265,235,217,290,101,118,92,90,103,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210103,871,711,472,439,390,302,194,300,223,333,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
# !jupyter nbconvert --to script SDC_bow제작.ipynb 